In [ ]:
# Install necessary packages
!pip install torch numpy pandas scikit-learn

# Import necessary libraries
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, accuracy_score
from torch.utils.data import DataLoader, Dataset
from model_reconstructed import UNet1D, AdvancedCNNAutoencoder

# Verify the Python environment
import sys
print(sys.executable)

In [ ]:
# Define dataset class
class SnoreDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return torch.tensor(feature, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# Define function to evaluate the models
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    predictions = np.vstack(all_preds)
    true_labels = np.vstack(all_labels)
    mse = mean_squared_error(true_labels, predictions)
    
    # For accuracy, you need to threshold the predictions and compare to true labels
    accuracy = accuracy_score(np.round(true_labels), np.round(predictions))
    return mse, accuracy

In [ ]:
# Load test features and labels
feature_sets = ['mfccs', 'spectral', 'rmse', 'zcr']
test_features = {key: np.load(f'../Dataset/test_{key}_features.npy') for key in feature_sets}
test_labels = np.load('../Dataset/test_labels.npy')

# Prepare the test datasets and dataloaders
test_datasets = {key: SnoreDataset(test_features[key], test_labels) for key in feature_sets}
test_loaders = {key: DataLoader(test_datasets[key], batch_size=16, shuffle=False) for key in feature_sets}

In [ ]:
# Load the trained models
# Load the trained models
unet_model = UNet1D()
unet_model.load_state_dict(torch.load('../Reconstructed/unet1d_model.pth'))

cnn_autoencoder_model = AdvancedCNNAutoencoder()
cnn_autoencoder_model.load_state_dict(torch.load('../Reconstructed/advanced_cnn_autoencoder_model.pth'))

In [ ]:
# Evaluate the models
results = []
for feature_set in feature_sets:
    unet_mse, unet_accuracy = evaluate_model(unet_model, test_loaders[feature_set])
    cnn_autoencoder_mse, cnn_autoencoder_accuracy = evaluate_model(cnn_autoencoder_model, test_loaders[feature_set])
    results.append({
        'Feature Set': feature_set,
        'Model': 'UNet1D',
        'MSE': unet_mse,
        'Accuracy': unet_accuracy
    })
    results.append({
        'Feature Set': feature_set,
        'Model': 'AdvancedCNNAutoencoder',
        'MSE': cnn_autoencoder_mse,
        'Accuracy': cnn_autoencoder_accuracy
    })

for result in results:
    print(f"Feature Set: {result['Feature Set']}, Model: {result['Model']}, MSE: {result['MSE']}, Accuracy: {result['Accuracy']}")

In [ ]:
# Create the performance table
df = pd.DataFrame(results)
print(df)